In [133]:
import os
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
import geopandas as gpd

rs = 123
np.random.seed(rs)
random.seed(rs)

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.linear_model import *
from sklearn.neighbors import KNeighborsRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.metrics import confusion_matrix

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import *
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xg

# 1. 데이터 불러오기

In [134]:
dataset_gid = gpd.read_file('model_dataset.geojson', driver= 'GeoJSON')
dataset_gid

,gid,acci_cnt,사망자수,중상자수,부상신고자수,일강수량(mm),일 최심적설(cm),안개 계속시간(hr),보행신호등,차량신호등,안전지대,횡단보도,도로속도,정차금지대,교통안전표지,CCTV,총인구,교통노드,건물면적,차량등록대수,전체_추정교통량,혼잡시간강도,geometry
0,다바866110,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,13,0,146.59,4,8283.000,2.74000,"MULTIPOLYGON (((127.35076 36.29622, 127.35076 ..."
1,다바823157,0,0.0,0.0,0.0,0.0,0.0,0.0,3,4,2,3,0,0,14,0,267,1,5944.98,0,1181.625,38.47125,"MULTIPOLYGON (((127.30277 36.33853, 127.30276 ..."
2,다바928260,0,0.0,0.0,0.0,0.0,0.0,0.0,6,5,0,4,3,0,14,0,78,2,1487.49,40,3783.750,70.80250,"MULTIPOLYGON (((127.41967 36.43152, 127.41967 ..."
3,다바931203,2,0.0,1.0,0.0,4.2,3.3,0.0,1,3,1,1,2,0,5,0,0,0,1291.19,409,11596.000,98.50000,"MULTIPOLYGON (((127.42307 36.38013, 127.42307 ..."
4,다바836109,0,0.0,0.0,0.0,0.0,0.0,0.0,2,0,0,2,0,0,6,0,396,0,12875.52,0,678.875,65.44500,"MULTIPOLYGON (((127.31735 36.29527, 127.31735 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12174,다바911108,1,0.0,1.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.00,0,286.500,94.86000,"MULTIPOLYGON (((127.40088 36.29447, 127.40088 ..."
12175,다바968288,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,2,0,0,0,0.00,0,2515.250,3.82500,"MULTIPOLYGON (((127.46429 36.45678, 127.46429 ..."
12176,다바906266,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,3,0,0,0,0.00,0,16135.250,5.13750,"MULTIPOLYGON (((127.39512 36.43691, 127.39512 ..."
12177,다바906193,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,2,1,0,0,0.00,0,14414.500,48.29500,"MULTIPOLYGON (((127.39521 36.37110, 127.39521 ..."


# 2. 모델링 데이터 셋 

In [135]:
dataset_encode = pd.DataFrame()
for col in tqdm(dataset_gid.columns):
    if type(dataset_gid[col][0]) == str:
        if col == 'gid':
            gid_lbe = LabelEncoder()
            gid_lbe.fit(dataset_gid[col])
            dataset_encode[col] = gid_lbe.transform(dataset_gid[col])
        else : 
            lbe = LabelEncoder()
            lbe.fit(dataset_gid[col])
            dataset_encode[col] = lbe.transform(dataset_gid[col])
    else:
        dataset_encode[col] = dataset_gid[col]
dataset_encode

100%|██████████| 23/23 [00:00<00:00, 411.26it/s]


,gid,acci_cnt,사망자수,중상자수,부상신고자수,일강수량(mm),일 최심적설(cm),안개 계속시간(hr),보행신호등,차량신호등,안전지대,횡단보도,도로속도,정차금지대,교통안전표지,CCTV,총인구,교통노드,건물면적,차량등록대수,전체_추정교통량,혼잡시간강도,geometry
0,3173,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,13,0,146.59,4,8283.000,2.74000,"MULTIPOLYGON (((127.35076 36.29622, 127.35076 ..."
1,507,0,0.0,0.0,0.0,0.0,0.0,0.0,3,4,2,3,0,0,14,0,267,1,5944.98,0,1181.625,38.47125,"MULTIPOLYGON (((127.30277 36.33853, 127.30276 ..."
2,8410,0,0.0,0.0,0.0,0.0,0.0,0.0,6,5,0,4,3,0,14,0,78,2,1487.49,40,3783.750,70.80250,"MULTIPOLYGON (((127.41967 36.43152, 127.41967 ..."
3,8682,2,0.0,1.0,0.0,4.2,3.3,0.0,1,3,1,1,2,0,5,0,0,0,1291.19,409,11596.000,98.50000,"MULTIPOLYGON (((127.42307 36.38013, 127.42307 ..."
4,1192,0,0.0,0.0,0.0,0.0,0.0,0.0,2,0,0,2,0,0,6,0,396,0,12875.52,0,678.875,65.44500,"MULTIPOLYGON (((127.31735 36.29527, 127.31735 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12174,6668,1,0.0,1.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.00,0,286.500,94.86000,"MULTIPOLYGON (((127.40088 36.29447, 127.40088 ..."
12175,11088,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,2,0,0,0,0.00,0,2515.250,3.82500,"MULTIPOLYGON (((127.46429 36.45678, 127.46429 ..."
12176,6254,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,3,0,0,0,0.00,0,16135.250,5.13750,"MULTIPOLYGON (((127.39512 36.43691, 127.39512 ..."
12177,6222,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,2,1,0,0,0.00,0,14414.500,48.29500,"MULTIPOLYGON (((127.39521 36.37110, 127.39521 ..."


In [136]:
dataset_encode.to_csv('dataset_encode.csv', header = True, index = False)

In [137]:
X = dataset_encode.drop(['acci_cnt','geometry'], axis=1)
y = dataset_encode['acci_cnt']

# 7:3 비율로 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 8888)

# 3. Model

## 3-1. Linear Regression

In [138]:
lm = LinearRegression()
lm.fit(X_train, y_train)
y_pred = lm.predict(X_train)

print('Train score')
print('R^2:',metrics.r2_score(y_train, y_pred))
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_train, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_train, y_pred))
print('MSE:',metrics.mean_squared_error(y_train, y_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_train, y_pred)))

y_test_pred = lm.predict(X_test)
acc_linreg = metrics.r2_score(y_test, y_test_pred)
print('\nTest score')
print('R^2:', acc_linreg)
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_test, y_test_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_test, y_test_pred))
print('MSE:',metrics.mean_squared_error(y_test, y_test_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))

Train score
R^2: 0.770262488478746
Adjusted R^2: 0.7696951019396485
MAE: 0.9932542331280609
MSE: 3.979825772166365
RMSE: 1.994950067587248

Test score
R^2: 0.8018689031271612
Adjusted R^2: 0.8007233213445815
MAE: 1.021563684539744
MSE: 4.089657137204347
RMSE: 2.022290072468425


## 3-2. Random Forest Regressor

In [139]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_train)

print('Train score')
print('R^2:',metrics.r2_score(y_train, y_pred))
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_train, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_train, y_pred))
print('MSE:',metrics.mean_squared_error(y_train, y_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_train, y_pred)))

y_test_pred = rf.predict(X_test)

acc_rf = metrics.r2_score(y_test, y_test_pred)
print('\nTest score')
print('R^2:', acc_rf)
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_test, y_test_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_test, y_test_pred))
print('MSE:',metrics.mean_squared_error(y_test, y_test_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))

Train score
R^2: 0.9720733381068625
Adjusted R^2: 0.9720043671672228
MAE: 0.30789560117302056
MSE: 0.4837836363636364
RMSE: 0.6955455674243323

Test score
R^2: 0.831633878907203
Adjusted R^2: 0.8306603963788581
MAE: 0.834367816091954
MSE: 3.4752732895457035
RMSE: 1.8642084887548667


In [140]:
f1 = pd.DataFrame({'data': X.columns, 'importance': rf.feature_importances_}).sort_values(by='importance', ascending=False)
f1.head(10)

,data,importance
2,중상자수,0.563954
4,일강수량(mm),0.203789
19,전체_추정교통량,0.034617
3,부상신고자수,0.024108
5,일 최심적설(cm),0.020154
20,혼잡시간강도,0.017993
13,교통안전표지,0.017244
17,건물면적,0.017159
0,gid,0.016709
8,차량신호등,0.013824


## 3-3. XGBRegressor

In [141]:
max_depth = [2, 3, 5, 10, 15]
booster=['gbtree','gblinear']
learning_rate=[0.05,0.1,0.15,0.20]
min_child_weight=[1,2,3,4]
base_score = [0.1,0.25,0.5]
n_estimators = [5, 10, 15, 20]


hyperparameter_grid = {'n_estimators': n_estimators,
                       'max_depth':max_depth,
                       'learning_rate':learning_rate,
                       'min_child_weight':min_child_weight,
                       'booster':booster,
                       'base_score':base_score}

regressor = XGBRegressor()
gd=RandomizedSearchCV(estimator=regressor,param_distributions=hyperparameter_grid,verbose=True)

gd.fit(X_train,y_train)

print(gd.best_score_)
print(gd.best_estimator_)
best_xgb = gd.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[01:28:22] WARNING: /workspace/src/learner.cc:480: 
Parameters: { max_depth, min_child_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[01:28:23] WARNING: /workspace/src/learner.cc:480: 
Parameters: { max_depth, min_child_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[01:28:23] WARNING: /workspace/src/learner.cc:480: 
Parameters: { max_depth, min_child_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   19.7s finished


In [142]:
best_xgb.fit(X_train,y_train)

y_pred = best_xgb.predict(X_train)
print('Train score')
print('R^2:',metrics.r2_score(y_train, y_pred))
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_train, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_train, y_pred))
print('MSE:',metrics.mean_squared_error(y_train, y_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_train, y_pred)))

y_test_pred = best_xgb.predict(X_test)
print('\nTest score')
acc_xgb = metrics.r2_score(y_test, y_test_pred)
print('R^2:', acc_xgb)
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_test, y_test_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_test, y_test_pred))
print('MSE:',metrics.mean_squared_error(y_test, y_test_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))

[01:28:39] WARNING: /workspace/src/learner.cc:480: 
Parameters: { max_depth, min_child_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Train score
R^2: 0.7616065251764522
Adjusted R^2: 0.7610177608613523
MAE: 0.9893682507625466
MSE: 4.129776146422978
RMSE: 2.0321850669717505

Test score
R^2: 0.7914893241723164
Adjusted R^2: 0.79028372830437
MAE: 1.0248727149062586
MSE: 4.30390376392696
RMSE: 2.0745852028603116


## 3-4. LightGBM

In [143]:
train = lgb.Dataset(X_train, label = y_train) 
test = lgb.Dataset(X_test, label = y_test)

base_score = [0.1,0.25,0.5]
booster=['gbtree','gblinear']
max_depth = [2, 3, 5, 10, 15]
min_child_weight=[1,2,3,4]
n_estimators = [100, 500, 900, 1100, 1500]
num_leaves = [32, 64]

params = {'base_score':base_score,
          'booster':booster,
          'max_depth':max_depth,
          'min_child_weight':min_child_weight,
          'n_estimators':n_estimators,
          'num_leaves':num_leaves
         }

regressor = lgb.LGBMRegressor()
gd=RandomizedSearchCV(estimator=regressor,param_distributions=hyperparameter_grid,verbose=True)
gd.fit(X_train,y_train)
print(gd.best_score_)
print(gd.best_estimator_)
best_lgb = gd.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[LightGBM] [Warning] Unknown parameter: booster
[LightGBM] [Warning] Unknown parameter: base_score
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: booster
[LightGBM] [Warning] Unknown parameter: base_score
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[LightGBM] [Warning] Unknown parameter: booster
[LightGBM] [Warning] Unknown parameter: base_score
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: booster
[LightGBM] [Warning] Unknown parameter: base_score
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: booster
[LightGBM] [Warning] Unknown parameter: base_score
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: booster
[LightGBM] [Warning] Unknown parameter: base_score
[LightGBM] [Warning] Unknown parameter: booster
[LightGBM] [Warning] Unknown parameter: base_score
[LightGBM] [Warning] Unknown parameter: booster
[LightGBM] [Warning] Unknown parameter: base_score
[LightGBM] [Warn

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   16.1s finished


[LightGBM] [Warning] Unknown parameter: booster
[LightGBM] [Warning] Unknown parameter: base_score
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
0.8047659461172184
LGBMRegressor(base_score=0.1, booster='gblinear', learning_rate=0.15,
              max_depth=5, min_child_weight=1, n_estimators=20)


In [144]:
best_lgb.fit(X_train,y_train)
y_pred = best_lgb.predict(X_train)
print('Train score')
print('R^2:',metrics.r2_score(y_train, y_pred))
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_train, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_train, y_pred))
print('MSE:',metrics.mean_squared_error(y_train, y_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_train, y_pred)))

y_test_pred = best_lgb.predict(X_test)

print('\nTest score')
acc_lgb = metrics.r2_score(y_test, y_test_pred)
print('R^2:', acc_lgb)
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_test, y_test_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_test, y_test_pred))
print('MSE:',metrics.mean_squared_error(y_test, y_test_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))

Train score
R^2: 0.8695483719599877
Adjusted R^2: 0.8692261934125527
MAE: 0.7712240371458751
MSE: 2.2598606028979753
RMSE: 1.5032832743358702

Test score
R^2: 0.8334061104953963
Adjusted R^2: 0.8324428749007936
MAE: 0.86818361253394
MSE: 3.438692360666651
RMSE: 1.8543711496533402


## 3-5. SVM Regressor

In [145]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [146]:
svma = svm.SVR()
svma.fit(X_train, y_train)
y_pred = svma.predict(X_train)

print('Train score')
print('R^2:',metrics.r2_score(y_train, y_pred))
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_train, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_train, y_pred))
print('MSE:',metrics.mean_squared_error(y_train, y_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_train, y_pred)))

y_test_pred = svma.predict(X_test)

acc_svm = metrics.r2_score(y_test, y_test_pred)
print('\nTest score')
print('R^2:', acc_svm)
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_test, y_test_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_test, y_test_pred))
print('MSE:',metrics.mean_squared_error(y_test, y_test_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))

Train score
R^2: 0.5361799025722268
Adjusted R^2: 0.5350343983918219
MAE: 0.9083503030971797
MSE: 8.034922835059053
RMSE: 2.834593945357792

Test score
R^2: 0.43517021748272755
Adjusted R^2: 0.43190440651552964
MAE: 1.072532704281961
MSE: 11.658746092037653
RMSE: 3.4144906050592163


## 3-6. Voting

In [147]:
#Voting할 모델 선택
r1 = best_xgb 
r2 = RandomForestRegressor()
r3 = best_lgb
r4 = GradientBoostingRegressor()

vreg = VotingRegressor([('xgb', r1), ('rf', r2),('lgbm',r3),('gb',r4)],weights=[1,0.8,0.8,0.7])

vreg.fit(X_train,y_train)
y_pred = vreg.predict(X_train)
print('Train score')
print('R^2:',metrics.r2_score(y_train, y_pred))
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_train, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_train, y_pred))
print('MSE:',metrics.mean_squared_error(y_train, y_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_train, y_pred)))

y_test_pred = vreg.predict(X_test)
print('\nTest score')
acc_vot = metrics.r2_score(y_test, y_test_pred)
print('R^2:', acc_vot)
print('Adjusted R^2:',1 - (1-metrics.r2_score(y_test, y_test_pred))*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1))
print('MAE:',metrics.mean_absolute_error(y_test, y_test_pred))
print('MSE:',metrics.mean_squared_error(y_test, y_test_pred))
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))

[01:29:03] WARNING: /workspace/src/learner.cc:480: 
Parameters: { max_depth, min_child_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[LightGBM] [Warning] Unknown parameter: booster
[LightGBM] [Warning] Unknown parameter: base_score
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
Train score
R^2: 0.8950921203049265
Adjusted R^2: 0.8948330275760548
MAE: 0.6675372057735797
MSE: 1.817357037382013
RMSE: 1.3480938533284739

Test score
R^2: 0.8388661255324115
Adjusted R^2: 0.8379344594080118
MAE: 0.845454554744629
MSE: 3.3259912762947037
RMSE: 1.8237300447968454


## 3-7. 결과 종합

In [148]:
models = pd.DataFrame({
    'Model': ['Linear Regression', 'Random Forest', 'XGBoost', 'lightGBM','Support Vector Machines','Voting'],
    'R-squared Score': [acc_linreg*100, acc_rf*100, acc_xgb*100, acc_lgb*100,acc_svm*100,acc_vot*100]})
models.sort_values(by='R-squared Score', ascending=False)

,Model,R-squared Score
5,Voting,83.886613
3,lightGBM,83.340611
1,Random Forest,83.163388
0,Linear Regression,80.186890
2,XGBoost,79.148932
4,Support Vector Machines,43.517022


# 4. 머신러닝 결과 상위 100개 도출
가장 성능이 좋은 Voting 앙상블 사용

In [149]:
#가장 잘 나온 모델인 best_xgb로 예측
dataset_encode['pred']=  vreg.predict(X.values)

In [150]:
#라벨인코더로 돌린 GID 복원
dataset_encode['gid'] = gid_lbe.inverse_transform(dataset_encode['gid'])

In [151]:
dataset_encode

,gid,acci_cnt,사망자수,중상자수,부상신고자수,일강수량(mm),일 최심적설(cm),안개 계속시간(hr),보행신호등,차량신호등,안전지대,횡단보도,도로속도,정차금지대,교통안전표지,CCTV,총인구,교통노드,건물면적,차량등록대수,전체_추정교통량,혼잡시간강도,geometry,pred
0,다바866110,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,13,0,146.59,4,8283.000,2.74000,"MULTIPOLYGON (((127.35076 36.29622, 127.35076 ...",772.192615
1,다바823157,0,0.0,0.0,0.0,0.0,0.0,0.0,3,4,2,3,0,0,14,0,267,1,5944.98,0,1181.625,38.47125,"MULTIPOLYGON (((127.30277 36.33853, 127.30276 ...",306.573854
2,다바928260,0,0.0,0.0,0.0,0.0,0.0,0.0,6,5,0,4,3,0,14,0,78,2,1487.49,40,3783.750,70.80250,"MULTIPOLYGON (((127.41967 36.43152, 127.41967 ...",480.416976
3,다바931203,2,0.0,1.0,0.0,4.2,3.3,0.0,1,3,1,1,2,0,5,0,0,0,1291.19,409,11596.000,98.50000,"MULTIPOLYGON (((127.42307 36.38013, 127.42307 ...",1174.257084
4,다바836109,0,0.0,0.0,0.0,0.0,0.0,0.0,2,0,0,2,0,0,6,0,396,0,12875.52,0,678.875,65.44500,"MULTIPOLYGON (((127.31735 36.29527, 127.31735 ...",487.877955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12174,다바911108,1,0.0,1.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.00,0,286.500,94.86000,"MULTIPOLYGON (((127.40088 36.29447, 127.40088 ...",104.695811
12175,다바968288,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,2,0,0,0,0.00,0,2515.250,3.82500,"MULTIPOLYGON (((127.46429 36.45678, 127.46429 ...",339.036855
12176,다바906266,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,3,0,0,0,0.00,0,16135.250,5.13750,"MULTIPOLYGON (((127.39512 36.43691, 127.39512 ...",1493.164499
12177,다바906193,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,2,1,0,0,0.00,0,14414.500,48.29500,"MULTIPOLYGON (((127.39521 36.37110, 127.39521 ...",1343.551052


In [152]:
ml_result = gpd.GeoDataFrame(dataset_encode, geometry = 'geometry')

In [153]:
ml_result_sort = ml_result.sort_values(by='pred', ascending = False)

In [154]:
ml_result_sort['x(경도)'] = ml_result_sort['geometry'].centroid.x
ml_result_sort['y(위도)'] = ml_result_sort['geometry'].centroid.y

# 위 / 경도 기준 50m 반경 데이터 구하기
ml_result_sort['geometry'] = ml_result_sort['geometry'].centroid.buffer(0.0005)
ml_result_sort

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """
/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to 

,gid,acci_cnt,사망자수,중상자수,부상신고자수,일강수량(mm),일 최심적설(cm),안개 계속시간(hr),보행신호등,차량신호등,안전지대,횡단보도,도로속도,정차금지대,교통안전표지,CCTV,총인구,교통노드,건물면적,차량등록대수,전체_추정교통량,혼잡시간강도,geometry,pred,x(경도),y(위도)
4594,다바927275,1,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,241.02,0,64236.0,45.38,"POLYGON ((127.41960 36.44549, 127.41960 36.445...",5772.993068,127.419101,36.445493
2838,다바927276,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,111.24,0,64236.0,45.38,"POLYGON ((127.41960 36.44639, 127.41960 36.446...",5768.892097,127.419100,36.446395
9452,다바927274,1,0.0,2.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.00,0,64236.0,45.38,"POLYGON ((127.41960 36.44459, 127.41960 36.444...",5768.600433,127.419102,36.444591
11125,다바927273,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.00,0,64236.0,45.38,"POLYGON ((127.41960 36.44369, 127.41960 36.443...",5765.185147,127.419103,36.443690
8469,다바926278,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.00,0,60878.0,43.17,"POLYGON ((127.41848 36.44820, 127.41848 36.448...",5467.406383,127.417982,36.448197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8103,다바779093,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.00,0,64.5,89.68,"POLYGON ((127.25497 36.28119, 127.25497 36.281...",14.802493,127.254473,36.281188
10260,다바826193,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.00,0,49.0,0.00,"POLYGON ((127.30708 36.37144, 127.30708 36.371...",14.253051,127.306584,36.371438
9658,다바832099,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.00,0,16.0,0.00,"POLYGON ((127.31398 36.28670, 127.31397 36.286...",14.066332,127.313475,36.286704
9778,다바804077,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.00,0,0.0,0.00,"POLYGON ((127.28285 36.26682, 127.28285 36.266...",3.880788,127.282350,36.266817


In [155]:
import folium

#대전시 중앙으로 오게 설정
center = (36.35, 127.4)
zoom = 12

#folium.Map으로 지도 그리기
m = folium.Map(location = center, 
               zoom_start = zoom,
               tiles = 'http://api.vworld.kr/req/wmts/1.0.0/BB730805-E115-3140-9139-BB79B7E3B74F/Base/{z}/{y}/{x}.png',
               attr = '대전시'
              )

#geojson에서 받은 값중 상위 100개만 지도에 표시

red = {'fillColor': '#ff0000', 'color': '#ff0000'}


folium.GeoJson(ml_result_sort.head(100),name='test2', style_function=lambda x:red).add_to(m)



m